## 一 配置

In [ ]:
# ===================================================================
#                      模块一：全局配置与环境初始化
# ===================================================================
# 这个单元格是整个字幕生成流程的控制中心。
# 你可以在这里修改所有关键参数，而无需改动后续的功能函数代码。
# ===================================================================


# ===================================================================
# 1. 导入所有必需的库
# ===================================================================

# --- Python标准库 ---
import os
import sys
import json
import logging
import re
import time
import gc
from pathlib import Path
from datetime import timedelta
import subprocess

# --- 第三方核心库 ---
import torch
import whisper
import requests
from pydub import AudioSegment

# --- 第三方辅助库 ---
import librosa
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display


# ===================================================================
# 2. 日志配置
# ===================================================================
# 配置日志记录器，方便在长时任务中追踪进度和排查问题
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout 
)


# ===================================================================
# 3. 全局核心配置
# ===================================================================
logging.info("--- 初始化全局配置 ---")

# --- 3.1. 路径配置 ---
# 输入目录：存放所有待处理的视频文件 (.mp4, .mkv等)
INPUT_DIR = Path("GrandBlue")
# 工作目录：所有中间文件和最终产物都将保存在这里
WORKSPACE_DIR = Path("workspace")
# 临时文件目录：存放提取的音频、人声、VAD切块等
TEMP_DIR = WORKSPACE_DIR / "temp"
# 输出目录：存放最终生成的 .srt 字幕文件
OUTPUT_DIR = WORKSPACE_DIR / "output"

# --- 3.2. API与模型配置 ---
# !!! 重要: 请在此处填入您的真实密钥 !!!
DEEPSEEK_API_KEY = "sk-xxxxxxxxxxxxxxxxxxxxxx" # 务必替换

# --- 3.3. 处理流程参数 ---
# 音频预处理参数
AUDIO_PREPROCESS_PARAMS = {
    "sample_rate_for_demucs": 44100, # Demucs所需的高采样率
    "sample_rate_for_whisper": 16000 # Whisper和VAD所需的标准采样率
}

# VAD (语音活动检测) 参数
VAD_PARAMS = {
    'threshold': 0.125,               # VAD敏感度阈值，越低越敏感
    'min_silence_duration_ms': 120,   # 判定为静音的最小持续时间
    'min_speech_duration_ms': 80,     # 判定为语音的最小持续时间
    'speech_pad_ms': 220              # 在检测到的语音片段前后各添加的静音边距
}

# ASR (自动语音识别) 参数
ASR_PARAMS = {
    "model_name": "large-v3", # Whisper模型名称: base, small, medium, large, large-v2, large-v3
    "language": "ja"          # 识别的语言 (ja: 日语, en: 英语, zh: 中文) 可用语言见https://github.com/openai/whisper/blob/main/whisper/tokenizer.py
}

# 翻译参数
TRANSLATION_PARAMS = {
    "batch_size": 10,         # 每次API调用翻译的字幕行数
    "max_retries": 1          # API调用失败后的最大重试次数
}


# ===================================================================
# 4. 翻译Prompt模板
# ===================================================================
# 这是提供给大语言模型(LLM)的指令，用于指导其如何进行翻译。
# 一个好的Prompt对于翻译质量至关重要。
PROMPT_TEMPLATE = """
# 角色扮演
你是一位专业的、深谙日本动漫文化的日语字幕翻译家。你尤其擅长翻译风格夸张、充满笑点、颜艺丰富和大学生日常吐槽的喜剧动漫，比如《碧蓝之海》。

# 核心任务
你的任务是将我提供的日语字幕文本，逐行翻译成自然、地道、且极具表现力的简体中文。

# 术语表 (Glossary) - 必须严格遵守
以下是本作品《碧蓝之海》(Grand Blue)中的固定翻译，任何情况下都不能更改：
- **核心地点和团体:**
  - Grand Blue: Grand Blue (潜水商店名)
  - PaB (Peek a Boo): PaB (潜水社团)
  - 伊豆大学: 伊豆大学
- **主要角色:**
  - 北原伊織 (きたはら いおり): 北原伊织
  - 古手川千紗 (こてがわ ちさ): 古手川千纱 (注意是“纱”不是“奈”)
  - 古手川奈々華 (こてがわ ななか): 古手川奈奈华
  - 今村耕平 (いまむら こうへい): 今村耕平
  - 浜岡梓 (はまおか あずさ): 滨冈梓
  - 吉原愛菜 (よしわら あいな): 吉原爱菜
  - 寿竜次郎 (ことぶき りゅうじろう): 寿龙次郎
  - 時田信治 (ときた しんじ): 时田信治
- **关键物品与活动:**
  - スクーバダイビング: 水肺潜水
  - ウーロン茶: 乌龙茶 (核心梗！详见下方指南)
  - 水: 水 (核心梗！详见下方指南)
- **特定语气词/口头禅:**
  - ウェイ: 根据语境灵活翻译为“哟—!”、“Wassup!”、“燥起来!”等，体现年轻人聚会时的起哄感，避免生硬的“Wao”。
  - チェイサー: (在酒桌上) 解围酒，请勿翻译成“追赶者”。

# 翻译风格与核心梗指南
1.  **忠于原作搞笑精神**: 这是最高原则！翻译必须能传达出原作的喜剧效果和“有病”感。允许使用贴切的网络流行语、俏皮话和吐槽风格的句子，但要避免过度使用而显得尴尬。
2.  **“生命之水”梗**:
    - 当角色一本正经地谈论“乌龙茶”或“水”，但情景明显是喝酒时（例如，能点燃），必须在译文中体现出这个笑点。
    - 优先方案：直接翻译成“乌龙茶”或“水”，但在后面用括号或注释点明，如：**乌龙茶（可燃）**、**水（生命之水）**。
    - 备选方案：用引号强调，如：“乌龙茶”、“水”。
3.  **口语化与颜艺匹配**: 角色都是大学生，对话风格随意。请使用自然的现代汉语口语。当画面出现夸张的“颜艺”时，译文的语气和用词也要足够夸张，以达到“音画同步”的喜剧效果。
4.  **处理ASR错误**: 输入的日文由ASR生成，可能存在错误。例如，遇到长段重复的无意义字符（如 `あああああ`），请理解其为语气词并恰当截断，如翻译为“啊——！”即可，无需逐字翻译。遇到明显的识别错误，请基于上下文进行合理修正。

# 输出格式要求
请严格按照以下JSON格式输出，不要添加任何额外的解释、注释或markdown标记。
- 格式: 一个JSON数组，每个对象包含 "id" (从1开始的原始行号) 和 "translation" (翻译后的文本)。
- 示例输入: `1: こんにちは\\n2: これはテストです`
- 示例输出: `[ {{"id": 1, "translation": "你好"}}, {{"id": 2, "translation": "这是一个测试"}} ]`

# 待翻译内容
现在，请翻译以下内容：
{BATCH_CONTENT}
"""



# ===================================================================
# 5. 环境初始化与检查
# ===================================================================
logging.info("--- 开始环境初始化与检查 ---")

# --- 5.1. 设置环境变量 (可选) ---
# autodl用
# logging.info("正在为HuggingFace等服务设置网络代理...")
# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value
# logging.info(" > 网络代理已设置。")

# --- 5.2. 创建工作目录 ---
logging.info("正在创建所需的工作目录...")
INPUT_DIR.mkdir(exist_ok=True)
WORKSPACE_DIR.mkdir(exist_ok=True)
TEMP_DIR.mkdir(exist_ok=True)
OUTPUT_DIR.mkdir(exist_ok=True)
logging.info(f" > 工作目录已确认: {WORKSPACE_DIR.resolve()}")

# --- 5.3. 发现待处理的视频文件 ---
logging.info(f"正在从 '{INPUT_DIR}' 目录中搜索视频文件...")
supported_extensions = ['.mp4', '.mkv', '.avi', '.mov', '.webm', '.flv']
video_files = []
for ext in supported_extensions:
    # 使用 .glob() 方法查找文件
    video_files.extend(list(INPUT_DIR.glob(f"*{ext}")))

# 对文件进行排序，确保按顺序处理
video_files.sort()

if not video_files:
    logging.warning(f"在 '{INPUT_DIR}' 目录中未找到任何支持的视频文件。")
    logging.warning("请将您的 .mp4, .mkv 等文件放入该目录。")
else:
    logging.info(f"成功发现 {len(video_files)} 个视频文件待处理:")
    for video_path in video_files:
        logging.info(f"  - {video_path.name}")

# --- 5.4. 检查并设置计算设备 (CPU/GPU) ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
logging.info(f" > PyTorch 将使用设备: {DEVICE.upper()}")
if DEVICE == "cuda":
    logging.info(f" > GPU 名称: {torch.cuda.get_device_name(0)}")

# --- 5.5. 检查API密钥 ---
if "sk-xxxxxxxx" in DEEPSEEK_API_KEY or not DEEPSEEK_API_KEY:
    logging.warning(" > 警告: DeepSeek API 密钥似乎是占位符或未填写，翻译步骤将失败。")
else:
    logging.info(" > DeepSeek API 密钥已配置。")
    
logging.info("--- 配置与环境初始化完成，系统已就绪 ---")

2025-07-15 02:40:20 - INFO - --- 初始化全局配置 ---
2025-07-15 02:40:20 - INFO - --- 开始环境初始化与检查 ---
2025-07-15 02:40:20 - INFO -  > HuggingFace Token 已配置。
2025-07-15 02:40:20 - INFO - 正在创建所需的工作目录...
2025-07-15 02:40:20 - INFO -  > 工作目录已确认: /root/autodl-tmp/Sub_Gen/workspace
2025-07-15 02:40:20 - INFO - 正在从 'GrandBlue' 目录中搜索视频文件...
2025-07-15 02:40:20 - INFO - 成功发现 12 个视频文件待处理:
2025-07-15 02:40:20 - INFO -   - S02E01.mp4
2025-07-15 02:40:20 - INFO -   - S02E02.mp4
2025-07-15 02:40:20 - INFO -   - S02E03.mp4
2025-07-15 02:40:20 - INFO -   - S02E04.mp4
2025-07-15 02:40:20 - INFO -   - S02E05.mp4
2025-07-15 02:40:20 - INFO -   - S02E06.mp4
2025-07-15 02:40:20 - INFO -   - S02E07.mp4
2025-07-15 02:40:20 - INFO -   - S02E08.mp4
2025-07-15 02:40:20 - INFO -   - S02E09.mp4
2025-07-15 02:40:20 - INFO -   - S02E10.mp4
2025-07-15 02:40:20 - INFO -   - S02E11.mp4
2025-07-15 02:40:20 - INFO -   - S02E12.mp4
2025-07-15 02:40:20 - INFO -  > PyTorch 将使用设备: CUDA
2025-07-15 02:40:20 - INFO -  > GPU 名称: NVIDIA 

## 二 音频预处理

In [3]:
# ===================================================================
# 步骤 1, 2, 和 2.5 的功能函数
# ===================================================================

def extract_audio(video_path: Path, audio_path: Path) -> bool:
    """
    步骤1：从视频文件提取高质量音轨 (44.1kHz, 立体声, 16-bit PCM WAV)。

    这是为了给 Demucs 提供最佳的输入，以保证人声分离的质量。

    参数:
        video_path (Path): 输入的视频文件路径。
        audio_path (Path): 输出的WAV音频文件路径。

    返回:
        bool: 如果文件已存在或提取成功，返回 True；否则返回 False。
    """
    if audio_path.exists():
        logging.info(f"高质量音频文件已存在，跳过提取: {audio_path.name}")
        return True
    
    logging.info(f"为Demucs提取高质量音频: '{video_path.name}' -> '{audio_path.name}'")
    command = [
        "ffmpeg", "-i", str(video_path), 
        "-vn",                   # 去除视频
        "-acodec", "pcm_s16le", # 使用无压缩的16-bit PCM编码
        "-ar", "44100",          # 采样率设置为 44.1kHz
        "-ac", "2",              # 设置为立体声
        str(audio_path), "-y"     # 覆盖已存在的文件
    ]
    try:
        # 使用 capture_output=True 来捕获 stdout 和 stderr
        result = subprocess.run(command, check=True, capture_output=True, text=True, encoding='utf-8')
        logging.info(f"高质量音轨提取成功。大小: {audio_path.stat().st_size / 1e6:.2f} MB")
        return True
    except subprocess.CalledProcessError as e:
        # 记录 ffmpeg 的错误输出
        logging.error(f"提取 '{video_path.name}' 的音频时出错。FFmpeg输出:\n{e.stderr}")
        return False

def separate_vocals(full_audio_path: Path, temp_dir: Path, device: str) -> Path | None:
    """
    步骤2：使用Demucs从高质量音轨中分离人声。

    依赖 'htdemucs' 模型，它在人声分离方面表现出色。
    函数会返回分离出的人声文件路径。

    参数:
        full_audio_path (Path): 输入的高质量WAV文件路径 (应为44.1kHz立体声)。
        temp_dir (Path):      用于存放Demucs输出的临时目录。
        device (str):         计算设备，如 "cuda" 或 "cpu"。

    返回:
        Path | None: 如果成功，返回分离出的人声文件路径；否则返回 None。
    """
    # Demucs 会在输出目录下创建一个与模型同名的子目录，
    # 再在其中创建一个与输入音频文件名（不含扩展名）同名的子目录。
    # e.g., temp_dir/htdemucs/S02E01/vocals.wav
    audio_stem = full_audio_path.stem 
    vocals_path = temp_dir / "htdemucs" / audio_stem / "vocals.wav"
    
    if vocals_path.exists():
        logging.info(f"高质量人声文件已存在，跳过分离: {vocals_path.name}")
        return vocals_path

    logging.info(f"开始使用 Demucs 分离人声: {full_audio_path.name}")
    
    # 注意：demucs.separate 是一个命令行入口点，推荐使用 subprocess 调用
    demucs_command = [
        sys.executable, "-m", "demucs.separate",
        "-n", "htdemucs",          # 使用 htdemucs 模型
        "--two-stems", "vocals",   # 只分离人声和背景声
        "-d", device,              # 指定设备
        "--out", str(temp_dir),    # 指定顶层输出目录
        str(full_audio_path)
    ]
    try:
        subprocess.run(demucs_command, check=True, capture_output=True, text=True, encoding='utf-8')
        logging.info(f"人声分离完成。高质量人声文件: {vocals_path}")
        return vocals_path
    except subprocess.CalledProcessError as e:
        logging.error(f"Demucs 处理 '{full_audio_path.name}' 时失败。Demucs输出:\n{e.stderr}")
        return None

def optimize_audio_for_transcription(raw_vocals_path: Path, temp_dir: Path) -> Path | None:
    """
    步骤2.5 (新增): 将Demucs分离出的高质量人声WAV转换为轻量级版本。

    转换后的格式为 16kHz 采样率、单声道，这是VAD和Whisper模型的标准输入格式。
    此步骤能显著减小文件大小，加快后续处理速度。

    参数:
        raw_vocals_path (Path): Demucs输出的原始人声文件路径。
        temp_dir (Path):        存放优化后音频的目录。

    返回:
        Path | None: 如果成功，返回优化后的音频文件路径；否则返回 None。
    """
    # e.g., "S02E01" -> "S02E01_16k_mono.wav"
    # 我们基于原始视频的 stem 来命名，而不是基于 "vocals.wav"，以保持关联性。
    video_stem = raw_vocals_path.parent.name
    optimized_path = temp_dir / f"{video_stem}_16k_mono.wav"
    
    if optimized_path.exists():
        logging.info(f"优化后的人声文件已存在，跳过转换: {optimized_path.name}")
        return optimized_path

    logging.info(f"开始优化人声文件用于转录: {raw_vocals_path.name}")
    logging.info(f"原始大小: {raw_vocals_path.stat().st_size / 1e6:.2f} MB")
    
    command = [
        "ffmpeg", "-i", str(raw_vocals_path),
        "-ar", "16000",          # 采样率降至 16kHz
        "-ac", "1",              # 通道转为单声道
        "-c:a", "pcm_s16le",     # 保持为WAV编码
        str(optimized_path),
        "-y"                     # 覆盖已存在的文件
    ]
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True, encoding='utf-8')
        logging.info(f"优化完成！新文件: {optimized_path.name}")
        logging.info(f"优化后大小: {optimized_path.stat().st_size / 1e6:.2f} MB")
        return optimized_path
    except subprocess.CalledProcessError as e:
        logging.error(f"优化人声文件时出错。FFmpeg输出:\n{e.stderr}")
        return None

# ===================================================================
# 测试区 (保持不变，用于独立验证本模块功能)
# ===================================================================
logging.info("--- [测试开始] 运行优化后的步骤1, 2, 和 2.5 ---")

if not video_files:
    logging.warning("未找到任何视频文件，无法执行音频预处理测试。")
else:
    # 使用第一个发现的视频进行测试
    test_video_path = video_files[0]
    # 从 "S02E01.mp4" 得到 "S02E01"
    test_video_stem = test_video_path.stem
    logging.info(f"将使用第一个视频进行测试: '{test_video_path.name}'")
    
    # 动态定义测试所需的文件路径
    # 1. 完整的高质量音频，作为Demucs的输入
    #    例如: workspace/temp/S02E01_full_audio.wav
    test_full_audio_path = TEMP_DIR / f"{test_video_stem}_full_audio.wav"

    # --- 步骤 1: 提取音频 ---
    if extract_audio(test_video_path, test_full_audio_path):
        
        # --- 步骤 2: 分离人声 ---
        # 传入 TEMP_DIR 作为demucs的输出目录
        raw_vocals_path = separate_vocals(test_full_audio_path, TEMP_DIR, DEVICE)
        
        if raw_vocals_path:
            
            # --- 新增步骤 2.5: 优化音频 ---
            # 同样传入 TEMP_DIR 作为优化后文件的存放目录
            optimized_vocals_path = optimize_audio_for_transcription(raw_vocals_path, TEMP_DIR)

            if optimized_vocals_path:
                logging.info("--- [测试成功] ---")
                logging.info("所有阶段（提取、分离、优化）均按预期工作或跳过。")
                logging.info(f"后续步骤将使用这个优化文件: {optimized_vocals_path.name}")
                # logging.info("下方是优化后的人声预览:")
                # display(Audio(optimized_vocals_path, autoplay=False))
            else:
                 logging.error("--- [测试失败] 在音频优化步骤 ---")
        else:
            logging.error("--- [测试失败] 在人声分离步骤 ---")
    else:
        logging.error("--- [测试失败] 在音频提取步骤 ---")

2025-07-15 02:18:07 - INFO - --- [测试开始] 运行优化后的步骤1, 2, 和 2.5 ---
2025-07-15 02:18:07 - INFO - 将使用第一个视频进行测试: 'S02E01.mp4'
2025-07-15 02:18:07 - INFO - 高质量音频文件已存在，跳过提取: S02E01_full_audio.wav
2025-07-15 02:18:07 - INFO - 高质量人声文件已存在，跳过分离: vocals.wav
2025-07-15 02:18:07 - INFO - 优化后的人声文件已存在，跳过转换: S02E01_full_audio_16k_mono.wav
2025-07-15 02:18:07 - INFO - --- [测试成功] ---
2025-07-15 02:18:07 - INFO - 所有阶段（提取、分离、优化）均按预期工作或跳过。
2025-07-15 02:18:07 - INFO - 后续步骤将使用这个优化文件: S02E01_full_audio_16k_mono.wav


## 三 VAD 静音检测

In [4]:
# ===================================================================
# 步骤 3 的功能函数
# ===================================================================

def detect_speech_segments(audio_path: Path, vad_params: dict) -> tuple[list, list] | tuple[None, None]:
    """
    在音频上运行 Silero VAD 以检测语音片段的时间戳。

    此函数加载 Silero VAD 模型，并使用指定的参数集来寻找语音活动。
    它会返回两种格式的时间戳：一种是基于采样点的精确格式（用于绘图），
    另一种是基于毫秒的通用格式（用于切割和保存）。

    参数:
        audio_path (Path):  待处理的音频文件路径 (必须是16kHz单声道WAV)。
        vad_params (dict):  一个包含VAD所需参数的字典，键应包括:
                            'threshold', 'min_silence_duration_ms',
                            'min_speech_duration_ms', 'speech_pad_ms'。

    返回:
        tuple[list, list] | tuple[None, None]: 
            如果成功，返回一个元组，包含 (采样点时间戳列表, 毫秒时间戳列表)。
            如果失败，返回 (None, None)。
    """
    SAMPLING_RATE = 16000 # Silero VAD 固定使用 16kHz
    logging.info("正在加载 Silero VAD 模型...")
    try:
        # force_reload=False 会使用缓存，避免重复下载
        model, utils = torch.hub.load(
            repo_or_dir='snakers4/silero-vad', model='silero_vad', force_reload=False
        )
        (get_speech_timestamps, _, read_audio, _, _) = utils
    except Exception as e:
        logging.error(f"加载 Silero VAD 模型失败，请检查网络或缓存: {e}", exc_info=True)
        return None, None

    logging.info(f"正在读取音频文件用于 VAD: {audio_path.name}")
    try:
        wav_tensor = read_audio(str(audio_path), sampling_rate=SAMPLING_RATE)
    except Exception as e:
        logging.error(f"读取音频文件 {audio_path.name} 失败: {e}", exc_info=True)
        return None, None
    
    logging.info(f"开始使用 Silero VAD (参数: {vad_params}) 检测语音时间戳...")
    
    # --- 使用从配置中传入的参数 ---
    try:
        speech_timestamps_samples = get_speech_timestamps(
            wav_tensor, model, sampling_rate=SAMPLING_RATE, **vad_params
        )
    except Exception as e:
        logging.error(f"VAD处理过程中发生错误: {e}", exc_info=True)
        return None, None
    
    # --- 将采样点格式 (samples) 转换为毫秒格式 (ms) ---
    speech_timestamps_ms = []
    for segment in speech_timestamps_samples:
        start_ms = round(segment['start'] / (SAMPLING_RATE / 1000))
        end_ms = round(segment['end'] / (SAMPLING_RATE / 1000))
        speech_timestamps_ms.append({'start': start_ms, 'end': end_ms})
        
    logging.info(f"VAD 检测到 {len(speech_timestamps_samples)} 个语音片段。")
    return speech_timestamps_samples, speech_timestamps_ms

def cut_audio_by_timestamps(original_audio_path: Path, timestamps_ms: list, output_chunk_dir: Path) -> Path | None:
    """
    根据VAD检测到的毫秒级时间戳，将长音频切割成多个小音频块。

    每个音频块的文件名都包含其在原始音频中的起止时间，方便后续处理和追溯。

    参数:
        original_audio_path (Path): 待切割的源音频文件路径 (16kHz, 单声道 .wav)。
        timestamps_ms (list):       包含 {'start': ms, 'end': ms} 字典的列表。
        output_chunk_dir (Path):    保存切割后音频块的目标目录。

    返回:
        Path | None: 如果成功，返回创建的音频块目录路径；否则返回 None。
    """
    logging.info(f"开始根据VAD时间戳切割音频: {original_audio_path.name}")
    
    # 确保输出目录存在，如果不存在则创建
    output_chunk_dir.mkdir(parents=True, exist_ok=True)
    
    try:
        source_audio = AudioSegment.from_wav(original_audio_path)
    except Exception as e:
        logging.error(f"使用pydub加载音频文件 '{original_audio_path.name}' 失败: {e}", exc_info=True)
        return None

    # 获取视频的基本名称 (stem)，用于构造有意义的文件名
    # e.g., "S02E01_16k_mono" -> "S02E01"
    video_stem = original_audio_path.stem.replace('_16k_mono', '')

    for i, ts in enumerate(timestamps_ms):
        start_ms = ts['start']
        end_ms = ts['end']
        
        # 使用 pydub 进行切片
        audio_chunk = source_audio[start_ms:end_ms]
        
        # 构造具有高区分度的文件名
        # 格式: S02E01_chunk_0001_868ms_1724ms.wav
        chunk_filename = f"{video_stem}_chunk_{i+1:04d}_{start_ms}ms_{end_ms}ms.wav"
        chunk_path = output_chunk_dir / chunk_filename
        
        try:
            # 导出为 WAV 格式
            audio_chunk.export(chunk_path, format="wav")
        except Exception as e:
            logging.error(f"导出音频块 {chunk_filename} 失败: {e}", exc_info=True)
            # 即使单个文件失败，也继续处理下一个
            continue
            
    logging.info(f"成功切割并保存了 {len(timestamps_ms)} 个音频片段到目录: {output_chunk_dir}")
    return output_chunk_dir

def plot_vad_results(audio_path: Path, speech_timestamps_samples: list):
    """
    (可选的调试函数) 使用matplotlib可视化VAD的检测结果。

    在音频波形图上，用红色区域标出检测到的语音活动部分。
    此函数对于调试VAD参数非常有用。

    参数:
        audio_path (Path):                音频文件路径 (用于绘制波形)。
        speech_timestamps_samples (list): Silero VAD 返回的原始采样点时间戳列表。
    """
    logging.info("正在生成 VAD 结果的可视化图表...")
    SAMPLING_RATE = 16000
    try:
        waveform, sr = librosa.load(str(audio_path), sr=SAMPLING_RATE)
        time_axis = np.linspace(0, len(waveform) / sr, num=len(waveform))
        
        # 创建一个与波形等长的掩码数组
        vad_mask = np.zeros_like(waveform, dtype=bool)
        for segment in speech_timestamps_samples:
            # 将检测到的语音部分在掩码中标记为 True
            vad_mask[segment['start']:segment['end']] = True
            
        plt.style.use('seaborn-v0_8-whitegrid')
        fig, ax = plt.subplots(figsize=(20, 5), dpi=100) # 调整了尺寸以便查看
        
        ax.plot(time_axis, waveform, label='Audio Waveform', color='royalblue', linewidth=0.7)
        
        y_min, y_max = ax.get_ylim()
        ax.fill_between(time_axis, y_min, y_max, where=vad_mask, color='crimson', alpha=0.4, label='Voice Activity (VAD)')
        
        ax.set_title(f"VAD Speech Activity Result: {audio_path.name}", fontsize=16)
        ax.set_xlabel("Time (seconds)")
        ax.set_ylabel("Amplitude")
        ax.set_xlim(0, len(waveform) / sr)
        ax.legend(loc='upper right')
        plt.tight_layout()
        plt.show()
    except Exception as e:
        logging.error(f"生成 VAD 可视化图表时出错: {e}", exc_info=True)

# ===================================================================
# 测试区
# ===================================================================
logging.info("--- [测试开始] 步骤3：语音活动检测 (VAD) 与音频切割 ---\n")

# 检查上一步骤生成的关键变量 'optimized_vocals_path' 是否存在且有效
# 'in locals()' 检查变量是否在当前作用域定义
if 'optimized_vocals_path' in locals() and optimized_vocals_path is not None and optimized_vocals_path.exists():
    logging.info(f"将对文件进行 VAD 处理: {optimized_vocals_path.name}")
    
    # --- 步骤 3.1: 运行 VAD 并获取两种格式的时间戳 ---
    # 定义VAD参数，后续可以移到全局配置中
    vad_parameters = {
        'threshold': 0.125, 
        'min_silence_duration_ms': 120,
        'min_speech_duration_ms': 80, 
        'speech_pad_ms': 220
    }
    speech_timestamps_samples, speech_timestamps_ms = detect_speech_segments(optimized_vocals_path, vad_parameters)
    
    if speech_timestamps_ms:
        logging.info("--- [VAD成功] VAD 处理完成 ---")
        logging.info("前5个检测到的语音片段 (单位: 毫秒):")
        for ts in speech_timestamps_ms[:5]:
            logging.info(f"  - Start: {ts['start']:>7d} ms, End: {ts['end']:>7d} ms")
        
        # --- 步骤 3.2: 将【毫秒】时间戳保存到 JSON 文件 (用于存档和调试) ---
        video_stem = optimized_vocals_path.stem.replace('_16k_mono', '')
        vad_results_path = TEMP_DIR / f"{video_stem}_vad_timestamps_ms.json"
        logging.info(f"正在将 VAD 结果 (毫秒) 保存到文件: {vad_results_path.name}")
        with open(vad_results_path, 'w', encoding='utf-8') as f:
            json.dump(speech_timestamps_ms, f, indent=2, ensure_ascii=False)

        # --- 步骤 3.3: 根据时间戳切割音频 ---
        # 定义切割后音频块的专用输出目录
        # e.g., workspace/temp/S02E01_vad_chunks
        vad_chunks_dir = TEMP_DIR / f"{video_stem}_vad_chunks"
        # 调用新函数执行切割
        cut_audio_by_timestamps(optimized_vocals_path, speech_timestamps_ms, vad_chunks_dir)
            
        # --- 步骤 3.4: 使用【采样点】时间戳进行精确绘图 (可选，取消注释即可看到图表) ---
        # logging.info("准备生成VAD可视化图表...")
        # plot_vad_results(optimized_vocals_path, speech_timestamps_samples)

    else:
        logging.warning("--- [测试警告] VAD 未能检测到任何语音片段，或在处理中发生错误。---")
else:
    logging.error("--- [测试失败] 找不到上一步生成的 'optimized_vocals_path' 文件。")
    logging.error("请确保上一个单元格的测试已成功运行，并生成了有效的优化后人声文件。")

2025-07-15 02:18:20 - INFO - --- [测试开始] 步骤3：语音活动检测 (VAD) 与音频切割 ---

2025-07-15 02:18:20 - INFO - 将对文件进行 VAD 处理: S02E01_full_audio_16k_mono.wav
2025-07-15 02:18:20 - INFO - 正在加载 Silero VAD 模型...
2025-07-15 02:18:21 - INFO - 正在读取音频文件用于 VAD: S02E01_full_audio_16k_mono.wav


Using cache found in /root/.cache/torch/hub/snakers4_silero-vad_master


2025-07-15 02:18:21 - INFO - 开始使用 Silero VAD (参数: {'threshold': 0.125, 'min_silence_duration_ms': 120, 'min_speech_duration_ms': 80, 'speech_pad_ms': 220}) 检测语音时间戳...
2025-07-15 02:18:38 - INFO - VAD 检测到 387 个语音片段。
2025-07-15 02:18:38 - INFO - --- [VAD成功] VAD 处理完成 ---
2025-07-15 02:18:38 - INFO - 前5个检测到的语音片段 (单位: 毫秒):
2025-07-15 02:18:38 - INFO -   - Start:     868 ms, End:    1788 ms
2025-07-15 02:18:38 - INFO -   - Start:   47236 ms, End:   49724 ms
2025-07-15 02:18:38 - INFO -   - Start:   51844 ms, End:   52668 ms
2025-07-15 02:18:38 - INFO -   - Start:   55460 ms, End:   57084 ms
2025-07-15 02:18:38 - INFO -   - Start:   57796 ms, End:   58876 ms
2025-07-15 02:18:38 - INFO - 正在将 VAD 结果 (毫秒) 保存到文件: S02E01_full_audio_vad_timestamps_ms.json
2025-07-15 02:18:38 - INFO - 开始根据VAD时间戳切割音频: S02E01_full_audio_16k_mono.wav
2025-07-15 02:18:38 - INFO - 成功切割并保存了 387 个音频片段到目录: workspace/temp/S02E01_full_audio_vad_chunks


## 四 ASR&原文字幕生成

#### 4.1 使用VAD切分的音频进行识别(首选)

In [5]:
# ===================================================================
# 步骤 4 的功能函数 (使用 OpenAI-Whisper)
# ===================================================================

def load_whisper_model(model_name: str, device: str):
    """
    加载官方的 OpenAI-Whisper 模型。

    此函数负责下载并缓存指定的Whisper模型，并将其加载到指定设备上。

    参数:
        model_name (str): 要加载的Whisper模型名称 (例如 "large-v3", "medium", "base")。
        device (str):     计算设备，如 "cuda" 或 "cpu"。

    返回:
        whisper.model.Whisper | None: 加载成功的Whisper模型对象，或在失败时返回None。
    """
    logging.info(f"正在加载官方 Whisper 模型: '{model_name}'")
    logging.info(f"将使用设备: {device}")
    
    try:
        # whisper.load_model 会自动处理模型的下载和缓存
        model = whisper.load_model(model_name, device=device)
        logging.info(f"✅ Whisper 模型 '{model_name}' 加载成功。")
        return model
    except Exception as e:
        logging.error(f"❌ 加载官方 Whisper 模型 '{model_name}' 失败: {e}", exc_info=True)
        return None

def transcribe_audio_chunks(whisper_model, audio_chunks_dir: Path, language: str) -> list[dict]:
    """
    遍历指定目录中的所有音频片段，并使用Whisper模型逐个进行语音识别。

    函数会从每个音频块的文件名中解析出其在原始音频中的起止时间，
    并将识别结果与时间戳整合成一个结构化列表。

    参数:
        whisper_model:      已加载的Whisper模型对象。
        audio_chunks_dir (Path): 存放VAD切割出的所有WAV音频块的目录。
        language (str):     要识别的语言代码 (例如 'ja' 代表日语)。

    返回:
        list[dict]: 一个包含转录结果的列表。每个元素是一个字典，
                    格式为: {'start_ms': int, 'end_ms': int, 'text': str}。
    """
    logging.info(f"准备从目录 '{audio_chunks_dir.name}' 中逐个识别音频片段...")
    
    # 查找并排序所有.wav文件，确保按时间顺序处理
    chunk_paths = sorted(audio_chunks_dir.glob("*.wav"))
    if not chunk_paths:
        logging.warning(f"在目录 '{audio_chunks_dir.name}' 中未找到任何 .wav 音频片段。")
        return []

    logging.info(f"找到 {len(chunk_paths)} 个音频片段，开始进行语音识别...")
    
    transcription_results = []
    total_chunks = len(chunk_paths)
    
    for i, chunk_path in enumerate(chunk_paths):
        # 使用 atexit 风格的日志输出，避免刷屏，只在关键节点或出错时输出
        if (i + 1) % 50 == 0: # 每处理50个文件打印一次进度
            logging.info(f"  -> 进度: {i+1} / {total_chunks}")
            
        try:
            # --- 调用 Whisper 的 transcribe 方法 ---
            # 它返回一个包含所有信息的字典
            result = whisper_model.transcribe(
                str(chunk_path),
                language=language,
                fp16=torch.cuda.is_available() # 在GPU上自动使用fp16以加速
            )
            
            # 获取识别出的文本，并去除首尾空白
            text = result["text"].strip()

            if text:
                # 从文件名解析时间戳的逻辑，文件名格式: S02E01_chunk_0001_868ms_1724ms.wav
                parts = chunk_path.stem.split('_')
                start_ms = int(parts[-2].replace('ms', ''))
                end_ms = int(parts[-1].replace('ms', ''))
                
                transcription_results.append({
                    "start_ms": start_ms,
                    "end_ms": end_ms,
                    "text": text
                })

        except (IndexError, ValueError) as e:
            # 捕获因文件名格式不正确导致的解析失败
            logging.warning(f"无法从文件名 {chunk_path.name} 解析时间戳，已跳过。错误: {e}")
        except Exception as e:
            logging.error(f"❌ 处理文件 {chunk_path.name} 时发生严重错误，已跳过。", exc_info=True)

        finally:
            gc.collect() 
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    logging.info(f"✅ 完成所有片段的识别，共获得 {len(transcription_results)} 条有效转录。")
    return transcription_results

def format_time_for_srt(milliseconds: int) -> str:
    """
    将毫秒数值转换为 SRT 字幕标准的时间戳格式 'HH:MM:SS,ms'。

    参数:
        milliseconds (int): 时间的毫秒数。

    返回:
        str: 格式化后的时间戳字符串。
    """
    td = timedelta(milliseconds=milliseconds)
    # timedelta.seconds 只包含不足一天的秒数部分
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    # timedelta.microseconds 包含的是秒以下部分的微秒数
    millis = td.microseconds // 1000
    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{millis:03d}"

def generate_srt_from_transcriptions(transcriptions: list, srt_output_path: Path):
    """
    将包含时间戳和文本的转录结果列表，生成为标准的 SRT 字幕文件。

    函数会先按起始时间对所有字幕条目进行排序，然后逐条写入文件。

    参数:
        transcriptions (list):  包含 {'start_ms', 'end_ms', 'text'} 的字典列表。
        srt_output_path (Path): 输出的 .srt 文件路径。
    """
    logging.info(f"正在生成 SRT 字幕文件到: {srt_output_path}")
    
    # 确保是按时间顺序写入的
    transcriptions.sort(key=lambda x: x['start_ms'])
    
    try:
        with open(srt_output_path, 'w', encoding='utf-8') as f:
            for i, res in enumerate(transcriptions, start=1):
                start_time_str = format_time_for_srt(res['start_ms'])
                end_time_str = format_time_for_srt(res['end_ms'])
                text = res['text']
                
                # 写入标准的SRT块格式
                f.write(f"{i}\n")
                f.write(f"{start_time_str} --> {end_time_str}\n")
                f.write(f"{text}\n\n") # 每个块后跟一个空行
                
        logging.info(f"✅ SRT 字幕文件已成功保存。")
    except Exception as e:
        logging.error(f"❌ 保存 SRT 文件时出错: {e}", exc_info=True)

# ===================================================================
# 测试区
# ===================================================================
logging.info("--- [测试开始] 步骤4：ASR语音识别 (使用Whisper) ---")

# 检查上一步骤生成的关键变量 'vad_chunks_dir' 是否存在且有效
if 'vad_chunks_dir' in locals() and vad_chunks_dir is not None and vad_chunks_dir.is_dir():
    
    # --- 模型加载 ---
    # 这些参数后续可以移到全局配置中
    WHISPER_MODEL_NAME = "large-v3"
    ASR_LANGUAGE = "ja"
    whisper_model = load_whisper_model(WHISPER_MODEL_NAME, DEVICE)
    
    if whisper_model:
        # --- 语音识别 ---
        transcription_data = transcribe_audio_chunks(whisper_model, vad_chunks_dir, ASR_LANGUAGE)
        
        if transcription_data:
            # --- 字幕文件生成 ---
            # 从VAD切块目录名中推断出视频的基本名称
            video_stem = vad_chunks_dir.name.replace('_vad_chunks', '')
            japanese_srt_path = OUTPUT_DIR / f"{video_stem}_ja.srt"
            
            generate_srt_from_transcriptions(transcription_data, japanese_srt_path)
            
            logging.info("\n--- [测试成功] 步骤4全部完成 ---")
            logging.info(f"生成的日语字幕文件位于: {japanese_srt_path}")
            logging.info("预览 (前5条):")
            for item in transcription_data[:5]:
                start = format_time_for_srt(item['start_ms'])
                end = format_time_for_srt(item['end_ms'])
                print(f"[{start} --> {end}] {item['text']}")
        else:
            logging.warning("--- [测试警告] ASR 未能从音频片段中识别出任何文本。---")
        
        # 显式释放模型占用的显存
        del whisper_model
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
else:
    logging.error("--- [测试失败] 找不到上一步生成的 'vad_chunks_dir' 目录。")
    logging.error("请确保上一个单元格的VAD测试已成功运行，并生成了音频块目录。")

2025-07-15 02:18:45 - INFO - --- [测试开始] 步骤4：ASR语音识别 (使用Whisper) ---
2025-07-15 02:18:45 - INFO - 正在加载官方 Whisper 模型: 'large-v3'
2025-07-15 02:18:45 - INFO - 将使用设备: cuda
2025-07-15 02:19:04 - INFO - ✅ Whisper 模型 'large-v3' 加载成功。
2025-07-15 02:19:04 - INFO - 准备从目录 'S02E01_full_audio_vad_chunks' 中逐个识别音频片段...
2025-07-15 02:19:04 - INFO - 找到 387 个音频片段，开始进行语音识别...
2025-07-15 02:19:37 - INFO -   -> 进度: 50 / 387
2025-07-15 02:20:07 - INFO -   -> 进度: 100 / 387
2025-07-15 02:20:36 - INFO -   -> 进度: 150 / 387
2025-07-15 02:21:04 - INFO -   -> 进度: 200 / 387
2025-07-15 02:21:35 - INFO -   -> 进度: 250 / 387
2025-07-15 02:22:01 - INFO -   -> 进度: 300 / 387
2025-07-15 02:22:30 - INFO -   -> 进度: 350 / 387
2025-07-15 02:23:36 - INFO - ✅ 完成所有片段的识别，共获得 373 条有效转录。
2025-07-15 02:23:36 - INFO - 正在生成 SRT 字幕文件到: workspace/output/S02E01_full_audio_ja.srt
2025-07-15 02:23:36 - INFO - ✅ SRT 字幕文件已成功保存。
2025-07-15 02:23:36 - INFO - 
--- [测试成功] 步骤4全部完成 ---
2025-07-15 02:23:36 - INFO - 生成的日语字幕文件位于: workspace/output/S02E

#### 4.x batch加速版（并非加速，弃用）

#### 4.2 混合策略(flag,To be improved)

#### 4.3 纯whisper(时间戳切分不准确)

## 五 翻译

In [ ]:
# ===================================================================
# 步骤 5 的功能函数
# ===================================================================

def parse_srt_file(srt_path: Path) -> list[dict]:
    """
    解析标准的 .srt 字幕文件。

    函数会读取文件内容，并使用正则表达式提取每个字幕条目的
    序号、时间戳和文本内容。

    参数:
        srt_path (Path): 输入的 .srt 文件路径。

    返回:
        list[dict]: 一个包含字幕信息的列表。每个元素是一个字典，
                    格式为: {'index': str, 'time': str, 'text': str}。
                    如果文件不存在或解析失败，返回空列表。
    """
    if not srt_path.exists():
        logging.error(f"SRT文件未找到: {srt_path}")
        return []
    try:
        with open(srt_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # 正则表达式匹配SRT块：序号、时间码、文本内容（可跨行）
        pattern = re.compile(r'(\d+)\n([\d:,]+ --> [\d:,]+)\n(.*?)\n\n', re.DOTALL)
        matches = pattern.findall(content)
        
        segments = [{'index': m[0], 'time': m[1], 'text': m[2].strip()} for m in matches]
        
        logging.info(f"成功从 '{srt_path.name}' 解析出 {len(segments)} 个字幕条目。")
        return segments
    except Exception as e:
        logging.error(f"解析 SRT 文件 '{srt_path.name}' 时出错: {e}", exc_info=True)
        return []

def translate_batch_deepseek(batch_content: str, api_key: str, prompt_template: str, max_retries: int = 1) -> list | None:
    """
    使用 DeepSeek API 批量翻译文本，并内置重试机制。

    函数会构建一个完整的prompt，调用DeepSeek API。如果发生网络错误或可恢复的
    API错误，它将自动重试指定次数。

    参数:
        batch_content (str):    待翻译的、格式化后的字幕批次文本。
        api_key (str):          您的 DeepSeek API 密钥。
        prompt_template (str):  包含 `{BATCH_CONTENT}` 占位符的完整Prompt模板。
        max_retries (int):      最大重试次数。默认为1，即总共尝试2次（初次+1次重试）。

    返回:
        list | None: 如果成功，返回一个包含翻译结果的JSON对象列表。
                     如果最终失败，返回 None。
    """
    if not api_key or "sk-" not in api_key:
        logging.error("DeepSeek API 密钥无效或未设置，无法翻译。")
        return None

    prompt = prompt_template.format(BATCH_CONTENT=batch_content)
    url = "https://api.deepseek.com/chat/completions"
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}
    data = {
        "model": "deepseek-chat",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.1, # 较低的温度确保翻译的稳定性和一致性
        "stream": False
    }
    
    # --- 重试逻辑 ---
    for attempt in range(max_retries + 1):
        try:
            response = requests.post(url, headers=headers, json=data, timeout=180) # 设置较长的超时
            response.raise_for_status() # 对 >=400 的状态码抛出HTTPError
            
            result_text = response.json()['choices'][0]['message']['content']
            
            # 尝试从返回的文本中提取干净的JSON数组
            # Deepseek有时会在JSON前后添加 "```json" 和 "```"
            json_match = re.search(r'\[.*\]', result_text, re.DOTALL)
            if json_match:
                return json.loads(json_match.group(0))
            else:
                logging.error(f"无法从API响应中解析出JSON数组: {result_text}")
                return None # 这是内容格式错误，重试可能无效，直接返回

        except requests.exceptions.Timeout:
            logging.warning(f"调用 DeepSeek API 超时 (尝试 {attempt + 1}/{max_retries + 1})。")
        except requests.exceptions.RequestException as e:
            logging.warning(f"调用 DeepSeek API 时发生网络错误 (尝试 {attempt + 1}/{max_retries + 1}): {e}")
        except (KeyError, IndexError, json.JSONDecodeError) as e:
            # 这些是响应格式错误，可能由API服务器问题导致，值得重试
            logging.warning(f"解析 DeepSeek API 响应时出错 (尝试 {attempt + 1}/{max_retries + 1}): {e}")

        if attempt < max_retries:
            logging.info("将在1秒后重试...")
            time.sleep(1) # 在重试前短暂等待
    
    logging.error("所有重试均失败，该批次翻译失败。")
    return None

def translate_srt_file(ja_srt_path: Path, zh_srt_output_path: Path, api_key: str, prompt_template: str, batch_size: int = 25):
    """
    读取日语SRT文件，分批次调用翻译API，并最终生成完整的中文SRT文件。

    如果某个批次翻译失败（包括重试后），将为该批次的所有字幕条目填充错误提示。

    参数:
        ja_srt_path (Path):        输入的日语SRT文件路径。
        zh_srt_output_path (Path): 输出的中文SRT文件路径。
        api_key (str):             DeepSeek API 密钥。
        prompt_template (str):     用于翻译的Prompt模板。
        batch_size (int):          每次API调用翻译的字幕条目数量。
    """
    logging.info(f"--- 开始翻译SRT文件: {ja_srt_path.name} ---")
    segments = parse_srt_file(ja_srt_path)
    if not segments: 
        logging.error("无法解析SRT文件或文件为空，翻译中止。")
        return

    all_translated_segments = {}
    total_segments = len(segments)

    for i in range(0, total_segments, batch_size):
        batch = segments[i:i+batch_size]
        # 将批次内的字幕格式化为 "ID: 文本" 的形式
        batch_text_input = "\n".join([f"{seg['index']}: {seg['text']}" for seg in batch])
        
        logging.info(f"  -> 正在翻译批次 [{i+1}-{min(i+batch_size, total_segments)}/{total_segments}]...")
        
        # 调用包含重试逻辑的翻译函数
        translated_batch = translate_batch_deepseek(batch_text_input, api_key, prompt_template)
        
        if translated_batch:
            for item in translated_batch:
                # API返回的id可能是数字或字符串，统一处理为字符串以匹配字典的键
                all_translated_segments[str(item['id'])] = item['translation']
            logging.info(f"  -> 批次 {i//batch_size + 1} 翻译成功。")
        else:
            # 如果翻译最终失败，为这个批次的所有条目填充占位符
            logging.error(f"批次 {i//batch_size + 1} 翻译失败，将使用占位符填充。")
            for seg in batch:
                all_translated_segments[seg['index']] = f"[翻译失败: {seg['text']}]"
        
        time.sleep(0.2) # 遵守API使用礼仪，在请求之间加入短暂延时

    # --- 重建SRT文件 ---
    final_srt_content = []
    for seg in segments:
        # 使用 .get() 安全地获取翻译文本，如果ID不存在则提供一个备用文本
        translated_text = all_translated_segments.get(seg['index'], f"[文本丢失: ID {seg['index']}]")
        final_srt_content.append(f"{seg['index']}\n{seg['time']}\n{translated_text}\n")

    try:
        # 使用 join 和 write 一次性写入，效率更高
        with open(zh_srt_output_path, 'w', encoding='utf-8') as f:
            f.write("\n".join(final_srt_content))
        logging.info(f"✅ 中文SRT文件已成功保存到: {zh_srt_output_path}")
    except IOError as e:
        logging.error(f"❌ 保存中文SRT文件时出错: {e}", exc_info=True)

# ===================================================================
# 测试区 
# ===================================================================
logging.info("--- [测试开始] 步骤5：字幕翻译 ---")

# 检查上一步骤生成的关键变量 'japanese_srt_path' 是否存在且有效
if 'japanese_srt_path' in locals() and japanese_srt_path is not None and japanese_srt_path.exists():
    
    # 动态生成中文SRT的输出路径
    video_stem = japanese_srt_path.stem.replace('_ja', '')
    chinese_srt_path = OUTPUT_DIR / f"{video_stem}_zh.srt"
    
    # 检查API密钥和Prompt模板是否已在全局配置中定义
    if 'DEEPSEEK_API_KEY' not in locals() or not DEEPSEEK_API_KEY or "sk-xxxxxxxx" in DEEPSEEK_API_KEY:
        logging.error("--- [测试失败] ---")
        logging.error("请在全局配置单元格中填入您的有效 DeepSeek API 密钥。")
    elif 'PROMPT_TEMPLATE' not in locals() or not PROMPT_TEMPLATE:
        logging.error("--- [测试失败] ---")
        logging.error("在全局配置单元格中未找到 'PROMPT_TEMPLATE'。")
    else:
        # 调用翻译主函数
        # BATCH_SIZE 可在全局配置中定义
        TRANSLATION_BATCH_SIZE = 10
        translate_srt_file(
            japanese_srt_path, 
            chinese_srt_path, 
            DEEPSEEK_API_KEY, 
            PROMPT_TEMPLATE,
            batch_size=TRANSLATION_BATCH_SIZE
        )
        
        logging.info("--- [测试完成] 步骤5执行完毕 ---")
        if chinese_srt_path.exists():
            try:
                with open(chinese_srt_path, 'r', encoding='utf-8') as f:
                    print("\n中文翻译文件预览 (前15行):")
                    preview_lines = [next(f) for _ in range(15) if f.tell() < f.seek(0, 2)]
                    f.seek(0) # 指针归位
                    print("".join(preview_lines))
            except Exception as e:
                logging.error(f"读取翻译预览失败: {e}")
else:
    logging.error("--- [测试失败] 找不到上一步生成的 'japanese_srt_path' 文件。")
    logging.error("请确保步骤4的ASR测试已成功运行，并生成了日语SRT文件。")

2025-07-15 02:40:26 - INFO - --- [测试开始] 步骤5：字幕翻译 ---
2025-07-15 02:40:26 - INFO - --- 开始翻译SRT文件: S02E01_full_audio_ja.srt ---
2025-07-15 02:40:26 - INFO - 成功从 'S02E01_full_audio_ja.srt' 解析出 373 个字幕条目。
2025-07-15 02:40:26 - INFO -   -> 正在翻译批次 [1-10/373]...
2025-07-15 02:40:48 - INFO -   -> 批次 1 翻译成功。
2025-07-15 02:40:48 - INFO -   -> 正在翻译批次 [11-20/373]...
2025-07-15 02:41:02 - INFO -   -> 批次 2 翻译成功。
2025-07-15 02:41:02 - INFO -   -> 正在翻译批次 [21-30/373]...
2025-07-15 02:41:15 - INFO -   -> 批次 3 翻译成功。
2025-07-15 02:41:15 - INFO -   -> 正在翻译批次 [31-40/373]...
2025-07-15 02:41:32 - INFO -   -> 批次 4 翻译成功。
2025-07-15 02:41:32 - INFO -   -> 正在翻译批次 [41-50/373]...
2025-07-15 02:41:48 - INFO -   -> 批次 5 翻译成功。
2025-07-15 02:41:48 - INFO -   -> 正在翻译批次 [51-60/373]...
2025-07-15 02:42:01 - INFO -   -> 批次 6 翻译成功。
2025-07-15 02:42:01 - INFO -   -> 正在翻译批次 [61-70/373]...
2025-07-15 02:42:16 - INFO -   -> 批次 7 翻译成功。
2025-07-15 02:42:17 - INFO -   -> 正在翻译批次 [71-80/373]...
2025-07-15 02:42:33 - INFO -   -> 批次 